<a href="https://colab.research.google.com/github/Zhennue/Final_Assignment_DAM101/blob/main/3rd_week.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!pip install torch torchvision gradio librosa numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os

def delete_non_npy_files(root_dir):
    for foldername, subfolders, filenames in os.walk(root_dir):
        for filename in filenames:
            if not filename.endswith('.npy'):
                file_path = os.path.join(foldername, filename)
                try:
                    os.remove(file_path)
                    print(f"Deleted: {file_path}")
                except Exception as e:
                    print(f"Error deleting {file_path}: {e}")

# Replace this with your directory path
root_directory = "/content/drive/MyDrive/SS2025/DAM101/Assignment_4/Melspectogram"
delete_non_npy_files(root_directory)


Deleted: /content/drive/MyDrive/SS2025/DAM101/Assignment_4/Melspectogram/Dzongkha/SONAM WNGDI TOP SONGS 2022 ｜｜ BHUTANESE NEW SONGS 2022｜｜_chunk5_clip1_10s_01.wav
Deleted: /content/drive/MyDrive/SS2025/DAM101/Assignment_4/Melspectogram/Dzongkha/PHUDOSA- Sonam Wangchen ft. Tashi Kheldon Dorji and The Monarchs @YeshiLhendupFilms ｜ Music Video_song6_clip1_10s_01.wav
Deleted: /content/drive/MyDrive/SS2025/DAM101/Assignment_4/Melspectogram/Dzongkha/Latest Hit Song ｜｜ Bhutanese Popular song_song3_clip3_10s_01.wav
Deleted: /content/drive/MyDrive/SS2025/DAM101/Assignment_4/Melspectogram/Dzongkha/Aum Penjor ｜ Full song  ｜ BTN Tshering Dorji ｜ Karma Jerry ｜ Tandin Bidha ｜ Ugyen Seldon ｜_part1_10s_01.wav
Deleted: /content/drive/MyDrive/SS2025/DAM101/Assignment_4/Melspectogram/Dzongkha/SONAM WNGDI TOP SONGS 2022 ｜｜ BHUTANESE NEW SONGS 2022｜｜_chunk4_clip1_10s_01.wav
Deleted: /content/drive/MyDrive/SS2025/DAM101/Assignment_4/Melspectogram/Dzongkha/PHUDOSA- Sonam Wangchen ft. Tashi Kheldon Dorji and 

In [41]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import numpy as np
import os
import json
from glob import glob
from tqdm import tqdm

# ==== CONFIG ====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_weights_path = "/content/drive/MyDrive/SS2025/DAM101/Assignment_4/best_model.pt"
val_data_path = "/content/drive/MyDrive/SS2025/DAM101/Assignment_4/Melspectogram"
num_classes = 4
expected_channels = 4
target_shape = (4, 128, 313)
batch_sizes = [4, 8, 16, 32]  # Tuning different batch sizes
results_output_path = "/content/drive/MyDrive/SS2025/DAM101/Assignment_4/tuning_results.json"

# ==== Dataset ====
class AudioFeatureDataset(Dataset):
    def __init__(self, root_dir):
        self.label_map = {'English': 0, 'Dzongkha': 1, 'Hindi': 2, 'Sharchogpa': 3}
        self.target_shape = target_shape
        self.filepaths = []

        for lang in self.label_map:
            lang_dir = os.path.join(root_dir, lang)
            if not os.path.isdir(lang_dir):
                continue
            for file in glob(os.path.join(lang_dir, "*.npy")):
                try:
                    data = np.load(file, allow_pickle=True)
                    if isinstance(data, np.ndarray) and data.ndim >= 2:
                        self.filepaths.append(file)
                except:
                    continue

    def __len__(self):
        return len(self.filepaths)

    def __getitem__(self, idx):
        filepath = self.filepaths[idx]
        data = np.load(filepath, allow_pickle=True)

        if data.ndim == 2:
            data = np.expand_dims(data, axis=0)
        elif data.ndim != 3:
            raise ValueError(f"Invalid shape: {filepath}, shape={data.shape}")

        if data.shape[0] < expected_channels:
            pad = np.zeros((expected_channels - data.shape[0], *data.shape[1:]))
            data = np.concatenate((data, pad), axis=0)
        elif data.shape[0] > expected_channels:
            data = data[:expected_channels]

        padded = np.zeros(self.target_shape)
        h, w = min(data.shape[1], self.target_shape[1]), min(data.shape[2], self.target_shape[2])
        padded[:, :h, :w] = data[:, :h, :w]

        label_name = os.path.basename(os.path.dirname(filepath))
        label = self.label_map[label_name]
        return torch.tensor(padded).float(), label

# ==== Model Definition ====
class ResNet34Custom(nn.Module):
    def __init__(self, in_channels=4, num_classes=4):
        super().__init__()
        base = models.resnet34(weights=None)
        base.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        base.fc = nn.Linear(base.fc.in_features, num_classes)
        self.net = base

    def forward(self, x):
        return self.net(x)

def load_model(path):
    model = ResNet34Custom(in_channels=expected_channels, num_classes=num_classes)
    state_dict = torch.load(path, map_location=device)
    if any(k.startswith("module.") for k in state_dict):
        state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
    model.load_state_dict(state_dict)
    model.to(device)
    model.eval()
    return model

def evaluate(model, dataloader):
    correct = 0
    total = 0
    for x, y in tqdm(dataloader, desc="Evaluating"):
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            outputs = model(x)
        preds = outputs.argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)
    return correct / total if total > 0 else 0.0

# ==== Run Tuning ====
def tune_evaluation_batch_size():
    val_dataset = AudioFeatureDataset(val_data_path)
    model = load_model(model_weights_path)

    results = {}
    for batch_size in batch_sizes:
        print(f"\nTrying batch size: {batch_size}")
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        acc = evaluate(model, val_loader)
        print(f"Accuracy @ batch_size={batch_size}: {acc:.4f}")
        results[batch_size] = acc

    # Save results
    with open(results_output_path, "w") as f:
        json.dump(results, f, indent=2)
    print(f"\nSaved results to: {results_output_path}")

if __name__ == "__main__":
    tune_evaluation_batch_size()


Trying batch size: 4


Evaluating: 100%|██████████| 4213/4213 [38:08<00:00,  1.84it/s]


Accuracy @ batch_size=4: 0.0783

Trying batch size: 8


Evaluating: 100%|██████████| 2107/2107 [37:17<00:00,  1.06s/it]


Accuracy @ batch_size=8: 0.0783

Trying batch size: 16


Evaluating: 100%|██████████| 1054/1054 [37:23<00:00,  2.13s/it]


Accuracy @ batch_size=16: 0.0783

Trying batch size: 32


Evaluating: 100%|██████████| 527/527 [38:02<00:00,  4.33s/it]

Accuracy @ batch_size=32: 0.0783

Saved results to: /content/drive/MyDrive/SS2025/DAM101/Assignment_4/tuning_results.json


In [50]:
!pip install --upgrade --force-reinstall gradio
!pip show gradio

  Using cached gradio-5.31.0-py3-none-any.whl.metadata (16 kB)
  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached ffmpy-0.5.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached gradio_client-1.10.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached groovy-0.1.2-py3-none-any.whl.metadata (6.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 5.5 MB/s eta 0:00:00
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached safehttpx-0.1.6-py3-none-any.whl.metadata (4.2 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Usi

Name: gradio
Version: 5.31.0
Summary: Python library for easily interacting with trained machine learning models
Home-page: https://github.com/gradio-app/gradio
Author: 
Author-email: Abubakar Abid <gradio-team@huggingface.co>, Ali Abid <gradio-team@huggingface.co>, Ali Abdalla <gradio-team@huggingface.co>, Dawood Khan <gradio-team@huggingface.co>, Ahsen Khaliq <gradio-team@huggingface.co>, Pete Allen <gradio-team@huggingface.co>, Ömer Faruk Özdemir <gradio-team@huggingface.co>, Freddy A Boulton <gradio-team@huggingface.co>, Hannah Blair <gradio-team@huggingface.co>
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: aiofiles, anyio, fastapi, ffmpy, gradio-client, groovy, httpx, huggingface-hub, jinja2, markupsafe, numpy, orjson, packaging, pandas, pillow, pydantic, pydub, python-multipart, pyyaml, ruff, safehttpx, semantic-version, starlette, tomlkit, typer, typing-extensions, uvicorn
Required-by: 


In [1]:
import torch
import torch.nn as nn
from torchvision import models
import gradio as gr
import numpy as np
import librosa
import json

# ========== Model Definition ==========
class ResNet34Custom(nn.Module):
    def __init__(self, in_channels=4, num_classes=4):
        super().__init__()
        base = models.resnet34(weights=None)
        base.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        base.fc = nn.Linear(base.fc.in_features, num_classes)
        self.net = base

    def forward(self, x):
        return self.net(x)

# ========== Language Classes ==========
languages = ['english', 'hindi', 'dzongkha', 'sharchogpa']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ========== Load Best Model ==========
model = ResNet34Custom(in_channels=4, num_classes=len(languages))
model_path = '/content/drive/MyDrive/SS2025/DAM101/Assignment_4/best_model.pt'
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# ========== Load Best Hyperparameters ==========
tuning_results_path = '/content/drive/MyDrive/SS2025/DAM101/Assignment_4/tuning_results.json'
with open(tuning_results_path, 'r') as f:
    best_hparams = json.load(f)

print(" Best Tuned Hyperparameters:")
print(json.dumps(best_hparams, indent=2))

# ========== Feature Extraction ==========
def extract_features(audio, sr=22050, n_mels=224, n_mfcc=224, hop_length=512):
    mel = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=n_mels, hop_length=hop_length)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length)
    chroma = librosa.feature.chroma_stft(y=audio, sr=sr, hop_length=hop_length)
    contrast = librosa.feature.spectral_contrast(y=audio, sr=sr, hop_length=hop_length)

    def pad_resize(feat):
        if feat.shape[1] < 224:
            pad_width = 224 - feat.shape[1]
            feat = np.pad(feat, ((0, 0), (0, pad_width)), mode='constant')
        return feat[:, :224]

    def normalize(x):
        x_min, x_max = x.min(), x.max()
        return (x - x_min) / (x_max - x_min + 1e-6)

    mel_db = normalize(pad_resize(mel_db))
    mfcc = normalize(pad_resize(mfcc))
    chroma = normalize(pad_resize(chroma))
    contrast = normalize(pad_resize(contrast))

    stacked = np.stack([mel_db, mfcc, chroma, contrast], axis=0).astype(np.float32)
    return stacked

# ========== Prediction Function ==========
def predict(audio):
    if audio is None:
        return "No audio provided"

    audio_np, sr = audio

    if sr != 22050:
        audio_np = librosa.resample(audio_np, orig_sr=sr, target_sr=22050)
        sr = 22050

    features = extract_features(audio_np, sr)
    x = torch.tensor(features).unsqueeze(0).to(device)

    with torch.no_grad():
        logits = model(x)
        probs = torch.softmax(logits, dim=1).cpu().numpy()[0]

    return {lang: float(prob) for lang, prob in zip(languages, probs)}

# ========== Gradio Interface ==========
iface = gr.Interface(
    fn=predict,
    inputs=gr.Audio(type="numpy"),  #  Only one audio input
    outputs=gr.Label(num_top_classes=4),
    title=" Language Classification from Audio",
    description=(
        " Record or upload a song/audio clip. "
        "The model predicts the language among: English, Hindi, Dzongkha, Sharchogpa.\n"
        f" Best Hyperparameters: {json.dumps(best_hparams)}"
    )
)

iface.launch()

 Best Tuned Hyperparameters:
{
  "4": 0.07827428639249896,
  "8": 0.07827428639249896,
  "16": 0.07827428639249896,
  "32": 0.07827428639249896
}
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bc0d04336f1cc0d0cd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
